In [25]:
# package imports
import pymongo
from pymongo import MongoClient
import pandas as pd
import pprint

In [26]:
# connect to client
client = MongoClient('localhost', 27017)

In [27]:
# check client connection
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [43]:
# make database
db = client.Library_Data

# make collections
books_collection = db.Books
authors_collection = db.Authors
patrons_collection = db.Patrons
loans_collection = db.Loans

In [29]:
# testing
test_query = {'Title': 'Painless Earth Science'}

In [30]:
search_result = books_collection.find(example_query)

In [31]:
for element in search_result:
    pprint.pprint(element)

{"Author's First Name": 'Edward',
 "Author's Last Name": 'Denecke',
 'Book_ID (Primary Key)': 28,
 'Genre': 'Non-fiction',
 'ISBN-10': '764146017',
 'Lexile': 'None',
 'Pages': 336,
 'Publisher': '\u200eBarrons Educational Services',
 'Rating': '3.8',
 'Title': 'Painless Earth Science',
 'Year': 2021,
 '_id': ObjectId('647662258f0203830646f0d9')}


In [62]:
all_loans = loans_collection.find({})

overdue_loans = []
for loan in all_loans:
    due_date = loan['Due_Date'].strftime('%Y-%m-%dT%H:%M:%SZ')
    return_date = loan['Return_Date'].strftime('%Y-%m-%dT%H:%M:%SZ')
    if return_date > due_date:
        overdue_loans.append(loan)

# len(overdue_loans) # 15
overdue_loans[0]

{'_id': ObjectId('64766af38f0203830646f10b'),
 'Loan_ID': 0,
 'Book_ID': 6,
 'Book Title': 'Harry Potter and the Prisoner of Azkaban',
 'Patron_ID': 1,
 'Patron First Name': 'Chuckleberry',
 'Patron Last Name': 'Gigglebottom',
 'Loan_Date': datetime.datetime(2022, 3, 14, 4, 0),
 'Due_Date': datetime.datetime(2022, 4, 14, 4, 0),
 'Return_Date': datetime.datetime(2022, 5, 14, 4, 0),
 'Fine': '$30.00'}

In [57]:
query = {
    'Book_ID': {'$gt': 0}
}

result = loans_collection.find(query)

for book in result:
    print(type(book))

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


In [45]:
result = loans_collection.aggregate([
  {
    '$group': {
      '_id': '$Book Title',
      'count': { '$sum': 1 }
    }
  },
  {
    '$sort': {
        'count': -1
    }
  }
])

most_popular_book = (next(result)['_id'], next(result)['count'])
most_popular_book

('Harry Potter and the Prisoner of Azkaban', 3)

In [46]:
result = loans_collection.aggregate([
  {
    '$group': {
      '_id': '$Book Title',
      'count': { '$sum': 1 }
    }
  },
  {
    '$sort': {
        'count': 1
    }
  }
])

least_popular_book = (next(result)['_id'], next(result)['count'])
least_popular_book

('Harry Potter and the Deathly Hallows', 1)